<a href="https://colab.research.google.com/github/bhumii-ka/google_lens_pro_max/blob/main/google_lens_pro_max.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gradio
!pip install pyngrok
!pip install transformers pillow requests

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
from PIL import Image
import requests
from io import BytesIO
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr
import torch
from pyngrok import ngrok

model_blip = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
processor_blip = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

access_token = "access_token"
model_llama = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", use_auth_token=access_token)
tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", use_auth_token=access_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  # Means the user did not define a `HF_TOKEN` secret => warn


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [12]:
# Initialize BLIP processor and model

# Function to process input (uploaded image and user question)
def process_input(uploaded_image, user_prompt):
    try:

        # Process the image directly (uploaded_image is already a PIL image)
        inputs_blip = processor_blip(images=uploaded_image, return_tensors="pt")

        # Generate the caption
        output = model_blip.generate(**inputs_blip)
        caption = processor_blip.decode(output[0], skip_special_tokens=True)

        # 2. Refine the search query using Llama model
        model_id = "meta-llama/Llama-3.2-1B-Instruct"
        pipe = pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto")

        # User message and system instructions
        messages = [
            {
              "role": "system",
              "content": "Process the image description and user's query, then return relevant web search results that align with the user's query specific needs. No need to give me the links just return one liner search query with all the user needs that can be directly put to browser in one single line only"
            },
            {
              "role": "user",
              "content": f"Image description: {caption} User's query: {user_prompt}"
            }
        ]

        # Generate refined query
        output_llama = pipe(messages, max_new_tokens=256)
        search_query = output_llama[0]["generated_text"][-1]['content']  # Get the refined query

        # 3. Perform the web search using Brave API
        # api_key = "SAkgKhqFS9TeHpQ3HW5FZDS_pumn_7"  # Replace with your actual Brave Search API key
        # headers = {
        #     'Authorization': f'Bearer {api_key}',
        #     'Content-Type': 'application/json'
        # }
        # params = {
        #     'q': search_query,  # Use the refined search query
        #     'count': 10,  # Number of results to return
        # }

        # # Request to Brave Search API
        # search_response = requests.get("https://api.brave.com/v1/search", headers=headers, params=params)
        # search_results = search_response.json()

        api_key = "api_key"  # Replace with your actual Zenserp API key
        params = {"q": search_query, "hl": "en", "apikey": api_key}
        search_response = requests.get("https://app.zenserp.com/api/v2/search", params=params)
        search_results = search_response.json()


        # Format the search results
        result_text = ""
        if 'organic' in search_results:
            for result in search_results['organic']:
                title = result.get('title')
                url = result.get('url')

                # Check if title and URL are not None or empty
                if title and url:  # Only add valid results
                    result_text += f"**[{title}]({url})**\n\n"
        else:
            result_text = "No search results found."

        return result_text, search_query, caption # Return the formatted results and refined query

    except Exception as e:
        return str(e), ""  # Return the error if something goes wrong

# Create Gradio interface
iface = gr.Interface(
    fn=process_input,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),  # Allow image upload via drag and drop
        gr.Textbox(lines=1, placeholder="Enter your query here...", label="User Question")
    ],
    outputs=[
        gr.Markdown(label= "Search Results", value="The results will be shown here..."),  # Output the search results
        gr.Textbox(label="Final Search Query"),  # Output the final refined query
        gr.Textbox(label="Image context")
    ],
    title="Image-Based Question Answering with Search Results",
    description="Upload an image and enter a question to get relevant search results along with the final query used."
)




In [14]:
ngrok.kill()

In [15]:
!ngrok config add-authtoken 2nNJp88nGU2SAJ2nuJf2qKQNPbR_5M4zZBqsQ9wRgsa6L7zHu
# Start a new ngrok tunnel
public_url = ngrok.connect(7860)  # Adjust the port number if necessary
print(f"Public URL: {public_url}")

# Launch the Gradio interface
iface.launch(share=True)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://6c4c-34-16-236-175.ngrok-free.app" -> "http://localhost:7860"
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3804cf967d418588a3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
